# **Importer les Bibliothèques et Initialiser Spark**

In [506]:
import sys
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
import seaborn as sns
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType

In [507]:
spark = SparkSession.builder \
    .appName("Data Processing dans catalogue csv") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()
print("Session Spark initialisée avec succès.")


Session Spark initialisée avec succès.


# **Charger et normaliser  les données depuis le fichier Hive**


In [508]:
# Charger les données depuis Hive
spark.sql("USE concessionnaire")
catalogue_df = spark.sql("SELECT * FROM catalogue_table_processed")

co2_df = spark.sql("SELECT * FROM co2_data_processed")


# Normaliser les colonnes de marque
co2_df = co2_df.withColumn("marque", F.lower(F.trim(F.col("marque"))))
catalogue_df = catalogue_df.withColumn("marque", F.lower(F.trim(F.col("marque"))))

co2_df = co2_df.withColumn("modele", F.lower(F.trim(F.col("modele"))))
catalogue_df = catalogue_df.withColumn("modele", F.lower(F.trim(F.col("modele"))))
# Vérifier les données chargées
print("Données Catalogue :")
catalogue_df.show(5)

print("Données CO2 :")
co2_df.show(1000)

Données Catalogue :
+----------+----------+-------------+-------------------+-----------+--------+--------+-------+--------+--------+
|    marque|    modele|modele_detail|unified_horse_power|   longueur|nbplaces|nbportes|couleur|occasion|    prix|
+----------+----------+-------------+-------------------+-----------+--------+--------+-------+--------+--------+
|   renault|    espace|         2.0T|                165|     longue|       7|       5|  rouge|    true| 21245.0|
|volkswagen|new beatle|          1.8|                110|    moyenne|       5|       5|   gris|   false| 26630.0|
|       bmw|        m5|             |                507|très longue|       5|       5|   bleu|    true| 66360.0|
|    nissan|    maxima|       3.0 V6|                200|très longue|       5|       5|  rouge|   false| 30000.0|
|  mercedes|      s500|             |                306|très longue|       5|       5|   noir|   false|101300.0|
+----------+----------+-------------+-------------------+-----------

# **Creation de **final_catalogue_df**  qui servira lors de la fusion des data  **

In [509]:
# Définir le schéma du DataFrame vide avec les bonnes colonnes
schema = StructType([
    StructField("marque", StringType(), True),
    StructField("modele", StringType(), True),
    StructField("modele_detail", StringType(), True),
    StructField("unified_horse_power", FloatType(), True),
    StructField("longueur", FloatType(), True),
    StructField("nbplaces", IntegerType(), True),
    StructField("nbportes", IntegerType(), True),
    StructField("couleur", StringType(), True),
    StructField("occasion", BooleanType(), True),
    StructField("prix", FloatType(), True),
    StructField("bonus_malus", FloatType(), True),
    StructField("cout_energie", FloatType(), True),
    StructField("rejets_co2", FloatType(), True)
])

# Créer le DataFrame vide avec ce schéma
final_catalogue_df = spark.createDataFrame([], schema)

# Afficher le DataFrame vide
final_catalogue_df.show(truncate=False)


+------+------+-------------+-------------------+--------+--------+--------+-------+--------+----+-----------+------------+----------+
|marque|modele|modele_detail|unified_horse_power|longueur|nbplaces|nbportes|couleur|occasion|prix|bonus_malus|cout_energie|rejets_co2|
+------+------+-------------+-------------------+--------+--------+--------+-------+--------+----+-----------+------------+----------+
+------+------+-------------+-------------------+--------+--------+--------+-------+--------+----+-----------+------------+----------+


# **Drop duplicate sur co2 et catalogue**

In [510]:
# Liste des colonnes à utiliser pour vérifier l'unicité (toutes sauf 'couleur')
columns_to_check = [col for col in catalogue_df.columns if col != "couleur"]

# Supprimer doublons dans le fichier CO2 au cas ou
co2_df = co2_df.dropDuplicates()

# Supprimer les doublons en se basant sur toutes les colonnes sauf 'couleur'
catalogue_df = catalogue_df.dropDuplicates(columns_to_check)

# Vérifier les résultats après suppression des doublons
print(f"Nombre de lignes uniques dans Co2 après suppression des doublons : {co2_df.count()}")
print(f"Nombre de lignes uniques dans Catalogue après suppression des doublons (sans 'couleur') : {catalogue_df.count()}")
catalogue_df.show(truncate=False)


Nombre de lignes uniques dans Co2 après suppression des doublons : 175
Nombre de lignes uniques dans Catalogue après suppression des doublons (sans 'couleur') : 54
+----------+----------+-------------+-------------------+--------+--------+--------+-------+--------+-------+
|marque    |modele    |modele_detail|unified_horse_power|longueur|nbplaces|nbportes|couleur|occasion|prix   |
+----------+----------+-------------+-------------------+--------+--------+--------+-------+--------+-------+
|volkswagen|polo      |1.2 6V       |55                 |courte  |5       |3       |blanc  |true    |8540.0 |
|volkswagen|polo      |1.2 6V       |55                 |courte  |5       |3       |blanc  |false   |12200.0|
|daihatsu  |cuore     |1.0          |58                 |courte  |5       |3       |noir   |false   |8850.0 |
|kia       |picanto   |1.1          |65                 |courte  |5       |5       |blanc  |false   |8990.0 |
|audi      |a2        |1.4          |75                 |courte  |

# **Fusionner les Données dans final_catalogue_df**

In [511]:
#fusionner les données
final_catalogue_df = final_catalogue_df.unionByName(catalogue_df, allowMissingColumns=True)

print(f"Nombre de lignes dans le catalogue final fusionné : {final_catalogue_df.count()}")
final_catalogue_df.show()

Nombre de lignes dans le catalogue final fusionné : 54
+----------+----------+-------------+-------------------+--------+--------+--------+-------+--------+-------+-----------+------------+----------+
|    marque|    modele|modele_detail|unified_horse_power|longueur|nbplaces|nbportes|couleur|occasion|   prix|bonus_malus|cout_energie|rejets_co2|
+----------+----------+-------------+-------------------+--------+--------+--------+-------+--------+-------+-----------+------------+----------+
|volkswagen|      polo|       1.2 6V|               55.0|  courte|       5|       3|  blanc|    true| 8540.0|       NULL|        NULL|      NULL|
|volkswagen|      polo|       1.2 6V|               55.0|  courte|       5|       3|  blanc|   false|12200.0|       NULL|        NULL|      NULL|
|  daihatsu|     cuore|          1.0|               58.0|  courte|       5|       3|   noir|   false| 8850.0|       NULL|        NULL|      NULL|
|       kia|   picanto|          1.1|               65.0|  courte|   

# **Modèles du catalogue absent de co2 et ajout des m**

In [512]:
from pyspark.sql import functions as F

# Étape 1 : Calculez les moyennes par marque dans co2_df
averages_by_marque = co2_df.groupBy("marque").agg(
    F.round(F.avg("bonus_malus"), 1).alias("avg_bonus_malus_by_marque"),
    F.round(F.avg("rejets_co2"), 1).alias("avg_rejets_co2_by_marque"),
    F.round(F.avg("cout_energie"), 1).alias("avg_cout_energie_by_marque")
)

# Étape 2 : Joignez final_catalogue_df avec co2_df pour obtenir les valeurs existantes
co2_df_select = co2_df.select(
    "marque",
    "modele",
    F.col("bonus_malus").alias("co2_bonus_malus"),
    F.col("rejets_co2").alias("co2_rejets_co2"),
    F.col("cout_energie").alias("co2_cout_energie")
)

final_with_co2 = final_catalogue_df.join(
    co2_df_select,
    on=["marque", "modele"],
    how="left"
)

# Étape 3 : Joignez avec les moyennes par marque
final_with_co2_and_averages = final_with_co2.join(
    averages_by_marque,
    on="marque",
    how="left"
)

# Étape 4 : Remplacez les valeurs manquantes par les valeurs correspondantes ou les moyennes
final_catalogue_df_enriched = final_with_co2_and_averages.withColumn(
    "bonus_malus",
    F.coalesce(
        final_with_co2_and_averages["bonus_malus"],  # Valeur originale
        final_with_co2_and_averages["co2_bonus_malus"],  # Valeur de co2_df
        final_with_co2_and_averages["avg_bonus_malus_by_marque"]  # Moyenne par marque
    )
).withColumn(
    "rejets_co2",
    F.coalesce(
        final_with_co2_and_averages["rejets_co2"],
        final_with_co2_and_averages["co2_rejets_co2"],
        final_with_co2_and_averages["avg_rejets_co2_by_marque"]
    )
).withColumn(
    "cout_energie",
    F.coalesce(
        final_with_co2_and_averages["cout_energie"],
        final_with_co2_and_averages["co2_cout_energie"],
        final_with_co2_and_averages["avg_cout_energie_by_marque"]
    )
)

# Étape 5 : Sélectionnez les colonnes finales
final_catalogue_df_enriched = final_catalogue_df_enriched.select(
    "marque", "modele", "modele_detail", "unified_horse_power",
    "longueur", "nbplaces", "nbportes", "couleur", "occasion", "prix",
    "bonus_malus", "rejets_co2", "cout_energie"
)

# Étape 6 : Affichez les résultats finaux
print("Catalogue final enrichi avec les moyennes de CO2 pour les modèles absents :")
final_catalogue_df_enriched.show(1000, truncate=False)

# Nombre de lignes dans le catalogue final
print(f"Nombre total de lignes dans le catalogue final enrichi : {final_catalogue_df_enriched.count()}")


Catalogue final enrichi avec les moyennes de CO2 pour les modèles absents :
+----------+----------+-------------+-------------------+-----------+--------+--------+-------+--------+--------+-----------+----------+------------+
|marque    |modele    |modele_detail|unified_horse_power|longueur   |nbplaces|nbportes|couleur|occasion|prix    |bonus_malus|rejets_co2|cout_energie|
+----------+----------+-------------+-------------------+-----------+--------+--------+-------+--------+--------+-----------+----------+------------+
|daihatsu  |cuore     |1.0          |58.0               |courte     |5       |3       |noir   |false   |8850.0  |NULL       |NULL      |NULL        |
|mercedes  |s500      |             |306.0              |très longue|5       |5       |rouge  |true    |70910.0 |7772.1     |171.2     |686.3       |
|skoda     |superb    |2.8 V6       |193.0              |très longue|5       |5       |blanc  |false   |31790.0 |-6000.0    |31.0      |85.0        |
|skoda     |superb    |2

# **Recalculcul des moyenne dasn final_df  et co2**


In [513]:
# Enregistrer final_catalogue_df comme vue temporaire
final_catalogue_df.createOrReplaceTempView("final_catalogue_temp_view")
co2_df.createOrReplaceTempView("co2_temp_view")


# Combiner les deux sources et calculer les moyennes
combined_averages_sql = spark.sql("""
    SELECT 
        ROUND(AVG(bonus_malus), 1) AS avg_bonus_malus,
        ROUND(AVG(rejets_co2), 1) AS avg_rejets_co2,
        ROUND(AVG(cout_energie), 1) AS avg_cout_energie
    FROM (
        SELECT bonus_malus, rejets_co2, cout_energie 
        FROM final_catalogue_temp_view
        WHERE bonus_malus IS NOT NULL AND rejets_co2 IS NOT NULL AND cout_energie IS NOT NULL
        UNION ALL
        SELECT bonus_malus, rejets_co2, cout_energie 
        FROM co2_temp_view
        WHERE bonus_malus IS NOT NULL AND rejets_co2 IS NOT NULL AND cout_energie IS NOT NULL
    ) combined_data
""")
combined_averages = combined_averages_sql.collect()[0]
print("Moyennes combinées (final_catalogue_df et co2_df) :", combined_averages)

print({
    "avg_bonus_malus": combined_averages["avg_bonus_malus"],
    "avg_rejets_co2": combined_averages["avg_rejets_co2"],
    "avg_cout_energie": combined_averages["avg_cout_energie"]
})

# Extraire les moyennes
avg_bonus_malus = combined_averages["avg_bonus_malus"]
avg_rejets_co2 = combined_averages["avg_rejets_co2"]
avg_cout_energie = combined_averages["avg_cout_energie"]
#drop temp view
spark.catalog.dropTempView("final_catalogue_temp_view")
spark.catalog.dropTempView("co2_temp_view")




Moyennes combinées (final_catalogue_df et co2_df) : Row(avg_bonus_malus=2554.7, avg_rejets_co2=121.5, avg_cout_energie=586.2)
{'avg_bonus_malus': 2554.7, 'avg_rejets_co2': 121.5, 'avg_cout_energie': 586.2}


True

# **Marques Absentes dans CO2**

In [514]:
from pyspark.sql import functions as F
print("Catalogue final enrichi avec les moyennes de CO2 pour les modèles absents :")
final_catalogue_df_enriched.show(1000, truncate=False)
# Étape 1 : Calculer les moyennes combinées (déjà fait)

# Étape 2 : Identifier les marques absentes dans co2_df (déjà fait)
# Obtenir la liste des marques absentes dans co2_df
marques_absentes_df = final_catalogue_df_enriched.select("marque").distinct().join(
    co2_df.select("marque").distinct(),
    on="marque",
    how="left_anti"
)
marques_absentes = [row["marque"] for row in marques_absentes_df.collect()]

print(f"Marques absentes dans CO2 : {len(marques_absentes)}")
print(marques_absentes)

# Étape 3 : Identifier les enregistrements à mettre à jour
unique_cols = ["marque", "modele", "modele_detail"]

models_to_update = final_catalogue_df_enriched.filter(
    (F.col("marque").isin(marques_absentes)) &
    (F.col("bonus_malus").isNull()) &
    (F.col("rejets_co2").isNull()) &
    (F.col("cout_energie").isNull())
)

# Étape 4 : Enrichir ces enregistrements avec les moyennes combinées
models_with_combined_averages = models_to_update.withColumn(
    "bonus_malus", F.lit(avg_bonus_malus)
).withColumn(
    "rejets_co2", F.lit(avg_rejets_co2)
).withColumn(
    "cout_energie", F.lit(avg_cout_energie)
)

# Étape 5 : Fusionner les enregistrements enrichis avec le reste du catalogue
keys_to_update = models_to_update.select(unique_cols).distinct()

final_catalogue_df_remaining = final_catalogue_df_enriched.join(
    keys_to_update,
    on=unique_cols,
    how='left_anti'
)

final_catalogue_df_updated = final_catalogue_df_remaining.unionByName(
    models_with_combined_averages,
    allowMissingColumns=True
)

# Étape 6 : Vérifier les résultats
print("Catalogue final après enrichissement des marques absentes avec les moyennes combinées :")
final_catalogue_df_updated.show(1000, truncate=False)

# Vérifier le nombre de valeurs NULL restantes
null_counts = final_catalogue_df_updated.select([
    F.count(F.when(F.col("bonus_malus").isNull(), True)).alias("bonus_malus_nulls"),
    F.count(F.when(F.col("rejets_co2").isNull(), True)).alias("rejets_co2_nulls"),
    F.count(F.when(F.col("cout_energie").isNull(), True)).alias("cout_energie_nulls")
])
null_counts.show()


Catalogue final enrichi avec les moyennes de CO2 pour les modèles absents :
+----------+----------+-------------+-------------------+-----------+--------+--------+-------+--------+--------+-----------+----------+------------+
|marque    |modele    |modele_detail|unified_horse_power|longueur   |nbplaces|nbportes|couleur|occasion|prix    |bonus_malus|rejets_co2|cout_energie|
+----------+----------+-------------+-------------------+-----------+--------+--------+-------+--------+--------+-----------+----------+------------+
|daihatsu  |cuore     |1.0          |58.0               |courte     |5       |3       |noir   |false   |8850.0  |NULL       |NULL      |NULL        |
|mercedes  |s500      |             |306.0              |très longue|5       |5       |rouge  |true    |70910.0 |7772.1     |171.2     |686.3       |
|skoda     |superb    |2.8 V6       |193.0              |très longue|5       |5       |blanc  |false   |31790.0 |-6000.0    |31.0      |85.0        |
|skoda     |superb    |2

In [515]:
spark.stop()

# **Vérifier les Données Fusionnées**

# **Enregistrer les Données Fusionnées**

In [516]:
# # Enregistrer les données dans Hive
# catalogue_with_co2.write.mode("overwrite").saveAsTable("catalogue_co2_merge_processed")
# print("Données fusionnées enregistrées avec succès.")

# **Analyser les Données Fusionnées**

In [517]:
# catalogue_with_co2.describe(["prix", "rejets_co2", "unified_horse_power", "longueur", "nbplaces"]).show()
# 
# # Vérification des véhicules avec rejets CO2 = 0
# zero_co2_vehicles = catalogue_with_co2.filter(F.col("rejets_co2") == 0)
# 
# # Compter le nombre total de véhicules avec rejets CO2 = 0
# zero_co2_count = zero_co2_vehicles.count()
# total_vehicles = catalogue_with_co2.count()
# 
# # Compter les véhicules avec rejets CO2 = 0 et un malus
# zero_co2_malus_count = zero_co2_vehicles.filter(F.col("bonus_malus") < 0).count()
# 
# # Calculer les proportions
# zero_co2_proportion = zero_co2_count / total_vehicles * 100
# zero_co2_malus_proportion = zero_co2_malus_count / zero_co2_count * 100 if zero_co2_count > 0 else 0
# 
# # Afficher les résultats
# print(f"Nombre total de véhicules : {total_vehicles}")
# print(f"Nombre de véhicules avec rejets CO2 = 0 : {zero_co2_count}")
# print(f"Proportion de véhicules avec rejets CO2 = 0 : {zero_co2_proportion:.2f}%")
# print(f"Nombre de véhicules avec rejets CO2 = 0 et malus : {zero_co2_malus_count}")
# print(f"Proportion des véhicules avec rejets CO2 = 0 et malus : {zero_co2_malus_proportion:.2f}%")
# 
# # Afficher les détails des incohérences
# print("Détails des véhicules avec rejets CO2 = 0 et malus :")
# zero_co2_vehicles.filter(F.col("bonus_malus") < 0).select(
#     "marque", "modele", "rejets_co2", "bonus_malus", "prix"
# ).show()
# 
# 
# 
# # Identifier les véhicules avec rejets CO2 = 0
# zero_co2_vehicles = catalogue_with_co2.filter(F.col("rejets_co2") == 0)
# 
# # Vérifier les marques concernées
# zero_co2_vehicles.groupBy("marque").count().show()
# 
# # Détail des modèles avec rejets CO2 = 0 et malus
# zero_co2_malus_vehicles = zero_co2_vehicles.filter(F.col("bonus_malus") < 0)
# zero_co2_malus_vehicles.select("marque", "modele", "rejets_co2", "bonus_malus").show(100)
# 


# **Données manquantes**

In [518]:
# from pyspark.sql.functions import col, sum, when
# null_counts = co2_df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in co2_df.columns])
# null_counts.show()


# **Distribution des Prix des Véhicules**

In [519]:
# prices = catalogue_with_co2.select("prix").rdd.flatMap(lambda x: x).collect()
# plt.hist(prices, bins=20, color='blue', edgecolor='black')
# plt.xlabel("Prix (€)")
# plt.ylabel("Fréquence")
# plt.title("Distribution des Prix des Véhicules")
# plt.grid(axis='y', linestyle='--')
# plt.show()


La majorité des véhicules ont un prix situé entre 20 000 € et 40 000 €, avec un pic autour de 25 000 €. Les véhicules de plus de 60 000 € sont peu nombreux, ce qui montre une distribution asymétrique centrée sur des véhicules de milieu de gamme.

# **Distribution de la Puissance des Véhicules**

In [520]:
# horsepower_data = catalogue_with_co2.select("unified_horse_power").rdd.flatMap(lambda x: x).collect()
# plt.hist(horsepower_data, bins=20, color='green', edgecolor='black')
# plt.xlabel("Puissance (ch)")
# plt.ylabel("Fréquence")
# plt.title("Distribution de la Puissance des Véhicules")
# plt.grid(axis='y', linestyle='--')
# plt.show()


La majorité des véhicules ont une puissance comprise entre 100 ch et 200 ch, avec un pic autour de 150 ch. Les véhicules de plus de 300 ch sont rares, ce qui indique une concentration sur des voitures de puissance moyenne.

# **Répartition des Marques**

In [521]:
# brand_distribution = catalogue_with_co2.groupBy("marque").count().toPandas()
# brand_distribution["percentage"] = (brand_distribution["count"] / brand_distribution["count"].sum()) * 100
# 
# plt.figure(figsize=(8, 8))
# plt.pie(brand_distribution["count"], startangle=90, colors=plt.cm.Paired.colors)
# labels = [f"{row['marque']} ({row['percentage']:.1f}%)" for _, row in brand_distribution.iterrows()]
# plt.legend(labels, title="Marques", loc="center left", bbox_to_anchor=(1, 0.5), fontsize=10)
# plt.title("Répartition des Marques")
# plt.tight_layout()
# plt.show()


# **Matrice de Corrélation**

In [522]:
# correlations = catalogue_with_co2.select("prix", "bonus_malus", "rejets_co2", "unified_horse_power").toPandas().corr()
# sns.heatmap(correlations, annot=True, cmap="coolwarm")
# plt.title("Matrice de Corrélation")
# plt.show()


Le prix est fortement corrélé à la puissance unifiée des véhicules (0.87), ce qui est attendu pour des voitures plus puissantes.
Les rejets de CO₂ sont très fortement corrélés au bonus/malus (0.94), reflétant leur impact direct sur la taxe écologique.
La puissance unifiée a une très faible corrélation négative avec les rejets de CO₂ (-0.09), probablement dû à la présence de véhicules électriques ou hybrides puissants mais peu polluants.

# **Relation entre Puissance et Prix**

In [523]:
# price_hp_data = catalogue_with_co2.select("prix", "unified_horse_power").rdd.map(tuple).collect()
# prices, hp = zip(*price_hp_data)
# plt.scatter(hp, prices, alpha=0.7, color='purple', edgecolor='black')
# plt.xlabel("Puissance (ch)")
# plt.ylabel("Prix (€)")
# plt.title("Relation entre Puissance et Prix des Véhicules")
# plt.grid(True)
# plt.show()


Le prix des véhicules augmente généralement avec la puissance, avec une tendance claire pour les puissances jusqu'à 200 ch. Cependant, au-delà de 300 ch, les prix varient fortement, reflétant la diversité des segments de véhicules, notamment les modèles haut de gamme

# **Relation entre Rejets CO2 et Bonus/Malus**

In [524]:
# df_pandas = catalogue_with_co2.select("rejets_co2", "bonus_malus").toPandas()
# sns.regplot(x="rejets_co2", y="bonus_malus", data=df_pandas, scatter_kws={"alpha": 0.6}, line_kws={"color": "red"})
# plt.xlabel("Rejets CO2 (g/km)")
# plt.ylabel("Bonus/Malus (€)")
# plt.title("Relation entre Rejets CO2 et Bonus/Malus")
# plt.grid(True)
# plt.show()


Une relation linéaire claire existe entre les rejets de CO₂ et le bonus/malus. Les véhicules avec des émissions élevées subissent un malus (valeurs négatives), tandis que ceux avec des émissions très faibles bénéficient d'un bonus (valeurs positives).

# **Analyse des Bonus/Malus**

In [525]:
# co2_df = catalogue_with_co2.withColumn("bonus_malus_category", 
#     F.when(F.col("bonus_malus") > 0, "Bonus")
#     .when(F.col("bonus_malus") < 0, "Malus")
#     .otherwise("Neutre ou Inconnu"))
# 
# co2_df.groupBy("bonus_malus_category").count().show()
# 
# # Visualisation
# bonus_dist = co2_df.groupBy("bonus_malus_category").count().toPandas()
# plt.pie(bonus_dist["count"], labels=bonus_dist["bonus_malus_category"], autopct="%1.1f%%")
# plt.title("Répartition des Catégories Bonus/Malus")
# plt.show()


# **Analyse des Marques**

In [526]:
# catalogue_with_co2.groupBy("marque").agg(F.avg("unified_horse_power").alias("avg_hp")).orderBy(F.desc("avg_hp")).show(10)


# **Valeurs aberrantes**

In [527]:
# from pyspark.sql.functions import mean, stddev
# 
# stats = catalogue_with_co2.select(mean("unified_horse_power").alias("mean"), stddev("unified_horse_power").alias("stddev")).first()
# mean_hp, stddev_hp = stats["mean"], stats["stddev"]
# 
# outliers = catalogue_with_co2.filter(
#     (F.col("unified_horse_power") > mean_hp + 3 * stddev_hp) |
#     (F.col("unified_horse_power") < mean_hp - 3 * stddev_hp)
# )
# print(f"Nombre de valeurs aberrantes : {outliers.count()}")
# outliers.select("marque", "modele", "unified_horse_power", "prix").show()


Parmi les valeurs aberrantes, on observe des véhicules comme la BMW M5, qui combine une puissance très élevée (507 ch) avec des prix particulièrement élevés (>60 000 €). Ces valeurs reflètent des modèles de luxe ou de performance qui se situent bien au-delà de la moyenne des données.

In [528]:
# # Ajouter une colonne de catégorie personnalisée
# co2_df = co2_df.withColumn(
#     "categorie_vehicule",
#     F.when(
#         (F.col("prix") < 20000) & 
#         (F.col("unified_horse_power") < 100) & 
#         (F.col("rejets_co2") < 50) & 
#         (F.col("bonus_malus") >= 0),
#         "Petite voiture économique"
#     ).when(
#         (F.col("longueur") > 4.5) & 
#         (F.col("nbplaces") >= 5) & 
#         (F.col("prix").between(30000, 50000)) & 
#         (F.col("unified_horse_power").between(150, 250)) & 
#         (F.col("rejets_co2").between(50, 150)),
#         "SUV familial"
#     ).when(
#         (F.col("prix") > 50000) & 
#         (F.col("unified_horse_power") > 300) & 
#         (F.col("rejets_co2") > 150) & 
#         (F.col("longueur") > 4.8),
#         "Voiture de luxe ou de sport"
#     ).when(
#         (F.col("unified_horse_power") > 200) & 
#         (F.col("rejets_co2") < 50) & 
#         (F.col("bonus_malus") > 0) & 
#         (F.col("prix") > 40000),
#         "Hybride ou électrique haut de gamme"
#     ).when(
#         (F.col("prix").between(20000, 40000)) & 
#         (F.col("unified_horse_power").between(100, 200)) & 
#         (F.col("rejets_co2").between(50, 150)),
#         "Véhicule polyvalent standard"
#     ).when(
#         (F.col("prix") > 80000) & 
#         (F.col("unified_horse_power") > 400) & 
#         (F.col("rejets_co2") > 150),
#         "Véhicule ultra-sportif"
#     ).when(
#         (F.col("prix") < 30000) & 
#         (F.col("unified_horse_power") < 150) & 
#         (F.col("rejets_co2") < 50) & 
#         (F.col("bonus_malus") > 0),
#         "Véhicule éco-responsable entrée de gamme"
#     ).otherwise("Non catégorisé")  # Catégorie par défaut si aucun critère n'est rempli
# )
# 
# # Afficher quelques exemples pour vérifier les catégories
# co2_df.select("marque", "modele", "prix", "unified_horse_power", "rejets_co2", "bonus_malus", "categorie_vehicule").show(200)


# **Fermer la session Spark**


In [529]:
# Arrêter la session Spark
# spark.stop()